# Compute single unit metrics for each session

1 - either use all annotated & uploaded ephys sessions as input or provide a list of session_ids

2 - choose whether to overwrite exsiting files

3 - set a savepath and filename for the output - one .pkl file per session

4 - compute metrics

(see description of metrics below)

In [5]:
import pandas as pd
import npc_lims
from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import spike_utils, data_utils
import dynamic_routing_analysis as dra
import os
import pickle
import upath

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#get all uploaded & annotated ephys sessions
ephys_sessions=tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True,))

KeyError: 'commit'

In [3]:
#alternatively, provide a list of session ids:
session_id_list=['712815_2024-05-22','712815_2024-05-20','708016_2024-05-01']
session_list=[]
for ss in session_id_list:
    session_list.append(npc_lims.get_session_info(ss))
ephys_sessions=tuple(session_list)

In [7]:
except_dict={}
overwrite_files=False
# save_path = dra.SINGLE_UNIT_METRICS_PATH
save_path= upath.UPath(r'D:\test_single_unit_metrics')

for session_info in ephys_sessions[:]:
    
    # try:
        trials=[]
        units=[]
        ##skip if file already exists
        
        if overwrite_files==False and (save_path / f"{session_info.id}_stim_context_modulation.pkl").exists():
            print('session',session_info.id,'already processed; skipping')
            continue
        lick_save_path=os.path.join(save_path,'lick_modulation')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # try:
        # session=DynamicRoutingSession(session_info.id)
        session_id=session_info.id
        trials=data_utils.load_trials_or_units(session_id, 'trials')
        units=data_utils.load_trials_or_units(session_id, 'units')

        # except Exception as e:
        #     print('failed to load trials or units, skipping session')
        #     except_dict[session_info.id]=e
        #     continue

        spike_utils.compute_stim_context_modulation(trials, units, session_info, save_path)

        spike_utils.compute_lick_modulation(trials, units, session_info, lick_save_path)

        print(session_info.id,'done')

    # except Exception as e:
    #     print(session_info.id,'failed')
    #     except_dict[session_info.id]=e


712815_2024-05-22_0 cached trials loaded
712815_2024-05-22_0 cached units loaded
712815_2024-05-22 done
712815_2024-05-20_0 cached trials not found, loading with npc_sessions
712815_2024-05-20_0 loading trials failed
712815_2024-05-20_0 cached units not found, loading with npc_sessions
712815_2024-05-20_0 loading units failed


TypeError: 'NoneType' object is not subscriptable

In [ ]:
except_dict

## single unit metrics descriptions


### baseline context modulation

comparisons between all visual context vs. auditory context trials

time window = [-0.1, 0] relative to stimulus onset

**'baseline_context_modulation_index'** - (vis-aud)/(vis+aud)

**'baseline_context_modulation_p_value'** - scipy.stats.mannwhitneyu(vis,aud)

**'baseline_context_modulation_zscore'** - (vis-aud)/std(all trials)

**'baseline_context_modulation_sign'** - np.sign(np.mean(vis-aud))

**'baseline_context_roc_auc'** - sklearn.metrics.roc_auc_score(context_label,baseline_frs)

**'linear_shift_baseline_context_true_value'** - mean (vis-aud) baseline fr difference, properly aligned

**'linear_shift_baseline_context_null_median'** - median (vis-aud) across all shifts

**'linear_shift_baseline_context_null_mean'** - mean (vis-aud) across all shifts

'**linear_shift_baseline_context_null_std**' - std of (vis-aud) across all shifts

'**linear_shift_baseline_context_p_value_higher**' - the fraction of shift values that the true value is higher than

'**linear_shift_baseline_context_p_value_lower**' - the fraction of shift values that the true value is lower than



### stimulus modulation

comparisons between baseline and stimulus firing rates per trial

baseline time window = [-0.1,0] sec relative to stimulus onset

stimulus time window = [0,0.1]

late stimulus time window = [0.1,0.2]

replace "stim" with specific stimulus name: vis1, vis2, sound1, sound2, or catch

'**stim_stimulus_modulation_index**' - (stim-baseline)/(stim+baseline)

'**stim_stimulus_modulation_zscore**' - (stim-baseline)/std(baseline)

'**stim_stimulus_modulation_p_value**' - scipy.stats.wilcoxon(stim,baseline)

'**stim_stimulus_modulation_sign**' - np.sign(np.mean(stim-baseline))

'**stim_stimulus_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(stim_vs_baseline_label,frs)

'**stim_stimulus_late_modulation_index**' - same as above but for later time window

'**stim_stimulus_late_modulation_zscore**' 

'**stim_stimulus_late_modulation_p_value**'

'**stim_stimulus_late_modulation_sign**'

'**stim_stimulus_late_modulation_roc_auc**'

'**stim_stim_latency**' - latency of the maximum response in the [0,0.3] time window - in units of binsize (default=0.025sec)



### context modulation of stimulus response

comparison between stimulus responses in same vs. other contexts

(i.e. for vis1, same context = vis, other context = aud)

time window = [0,0.1] relative to stimulus onset

'**stim_context_modulation_index**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_zscore**' - (stim_same_context-stim_other_context)/(stim_same_context+stim_other_context)

'**stim_context_modulation_sign**' - np.sign(np.mean(stim_same_context-stim_other_context))

'**stim_context_modulation_p_value**' - scipy.stats.mannwhitneyu(stim_same_context,stim_other_context)

'**stim_context_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(context_label,stim_frs)

'**stim_evoked_context_modulation_index**' - same as above but for evoked fr - subtract the [-0.1,0] baseline per trial

'**stim_evoked_context_modulation_zscore**'

'**stim_evoked_context_modulation_sign**'

'**stim_evoked_context_modulation_p_value**'



### lick modulation

comparisons between correct reject (no_lick) and false alarm (lick) trials

time window: [0.2,0.5]

baseline: [-0.5,-0.2]

'**lick_modulation_index**' - (lick-no_lick)/(lick+no_lick)

'**lick_modulation_zscore**' - (lick-no_lick)/std(baseline)

'**lick_modulation_p_value**' - st.mannwhitneyu(lick,no_lick)

'**lick_modulation_sign**' - np.sign(np.mean(lick-no_lick))

'**lick_modulation_roc_auc**' - sklearn.metrics.roc_auc_score(lick, no_lick)



### stimulus selectivity

comparisons between firing rate responses to different stimuli

time window: [0,0.1]

'**vis_discrim_roc_auc**' - vis1 vs. vis2

'**aud_discrim_roc_auc**' - sound1 vs. sound2

'**target_discrim_roc_auc**' - vis1 vs. sound1

'**nontarget_discrim_roc_auc**' - vis2 vs. sound2

'**vis_vs_aud**' - all vis vs. all sound


### trial response type selectivity

comparisons between response types, in different time windows

all stimuli included - maybe should be

time windows [0,0.1] [0.1,0.2] [0.2,0.3]

'cr_vs_fa_early_roc_auc'

'hit_vs_cr_early_roc_auc'

'hit_vs_fa_early_roc_auc'

'cr_vs_fa_mid_roc_auc'

'hit_vs_cr_mid_roc_auc'

'hit_vs_fa_mid_roc_auc'

'cr_vs_fa_late_roc_auc'

'hit_vs_cr_late_roc_auc'

'hit_vs_fa_late_roc_auc'
